In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import re
from datetime import date
import urllib.request as urllib2
import numpy as np
import json
from io import StringIO
import urllib.parse
import os

**======= basic entity record =========**

In [2]:
def cleanBaseEntityInfo(df):
    df.rename( columns={'Has id':'entity_name',
                        'Has name':'real_name',
                        'Has nationality':'nationality',
                        'Has team':'team',
                        'Has status':'player_status'}, inplace=True )
    
    col_list = ['Has twitch stream', 'Has twitter','Has facebook','Has youtube channel','Has esl profile']
    df[col_list] = df[col_list].fillna('None')
    # twitch;twitter;facebook;youtube;esl
    social_string = df['Has twitch stream'] + ';' + df['Has twitter'] + ';' + df['Has facebook'] + ';' + df['Has youtube channel'] + ';' + df['Has esl profile']
    df.insert(len(df.columns), 'social_links', social_string)
    # drop unnamed: 1 for NaN's and s-teir row as it is inconsistant and not clear what it is
    df.drop(columns=['Unnamed: 0','Has pagename','Has id sort','Has twitch stream', 'Has twitter','Has facebook','Has youtube channel','Has esl profile'], inplace = True)
    
    # move status column to end of the df
    column_to_move = df.pop("player_status")
    df.insert(4, "player_status", column_to_move)
    
    return df

In [3]:
def getAllEntityBaseInfo(link_list):
    all_entities_df_list = []
    for link in link_list:
        # Fetch page and stuff into dataframe
        temp_csv_df = pd.read_csv(link)
        clean_df = cleanBaseEntityInfo(temp_csv_df)
        all_entities_df_list.append(clean_df)
    all_entities_df = pd.concat(all_entities_df_list)
    # Add combined dataframe to master
    return all_entities_df.copy()

In [4]:
base_entity_link_1 = "https://liquipedia.net/apexlegends/Special:Ask/mainlabel%3D/format%3Dcsv/link%3Dnone/template%3DPortal-5Fplayers-2Fcountry-2Fplayer/sort%3DHas-20id-20sort/order%3Dasc/offset%3D0/limit%3D500/-5B-5BCategory:Players-5D-5D-20-5B-5B:%2B-5D-5D/-3FHas-20id/-3FHas-20name/-3FHas-20nationality/-3FHas-20team/-3FHas-20twitch-20stream/-3FHas-20twitter/-3FHas-20facebook/-3FHas-20pagename/-3FHas-20status/-3FHas-20youtube-20channel/-3FHas-20esl-20profile/-3FHas-20id-20sort/prettyprint%3Dtrue/unescape%3Dtrue/searchlabel%3DCSV"
base_entity_link_2 = "https://liquipedia.net/apexlegends/Special:Ask/mainlabel%3D/format%3Dcsv/link%3Dnone/template%3DPortal-5Fplayers-2Fcountry-2Fplayer/sort%3DHas-20id-20sort/order%3Dasc/offset%3D500/limit%3D500/-5B-5BCategory:Players-5D-5D-20-5B-5B:%2B-5D-5D/-3FHas-20id/-3FHas-20name/-3FHas-20nationality/-3FHas-20team/-3FHas-20twitch-20stream/-3FHas-20twitter/-3FHas-20facebook/-3FHas-20pagename/-3FHas-20status/-3FHas-20youtube-20channel/-3FHas-20esl-20profile/-3FHas-20id-20sort/prettyprint%3Dtrue/unescape%3Dtrue/searchlabel%3DCSV"
base_entity_link_3 = "https://liquipedia.net/apexlegends/Special:Ask/mainlabel%3D/format%3Dcsv/link%3Dnone/template%3DPortal-5Fplayers-2Fcountry-2Fplayer/sort%3DHas-20id-20sort/order%3Dasc/offset%3D1000/limit%3D500/-5B-5BCategory:Players-5D-5D-20-5B-5B:%2B-5D-5D/-3FHas-20id/-3FHas-20name/-3FHas-20nationality/-3FHas-20team/-3FHas-20twitch-20stream/-3FHas-20twitter/-3FHas-20facebook/-3FHas-20pagename/-3FHas-20status/-3FHas-20youtube-20channel/-3FHas-20esl-20profile/-3FHas-20id-20sort/prettyprint%3Dtrue/unescape%3Dtrue/searchlabel%3DCSV"
entity_links = [base_entity_link_1,base_entity_link_2,base_entity_link_3]

In [5]:
all_entity_info = getAllEntityBaseInfo(entity_links)
all_entity_info.to_csv(f'./data/entity_info.csv',index=False,quoting=csv.QUOTE_NONE,quotechar="",escapechar="\\")

In [6]:
all_entity_info[['real_name','real_name','team','player_status']]

,real_name,real_name,team,player_status
0,NaN,NaN,TSURUGI eSports,Active
1,Tessa Steis,Tessa Steis,NaN,Active
2,Gabriel Reinhardt,Gabriel Reinhardt,NaN,Active
3,Peter Benney,Peter Benney,NaN,Active
4,Sebastián Ezequiel Niz,Sebastián Ezequiel Niz,Pirate Dream Esports,Active
...,...,...,...,...
271,Jared Gitlin,Jared Gitlin,NaN,Retired
272,Jung-Da Lyu,Jung-Da Lyu,NaN,Retired
273,Mauro Londra,Mauro Londra,NaN,Retired
274,Salem Hassan Al Talhi,Salem Hassan Al Talhi,NaN,Active


In [7]:
all_entity_info['social_links']

0         None;https://twitter.com/OddKEI;None;None;None
1      https://www.twitch.tv/thousand_eyes;https://tw...
2      https://www.twitch.tv/1fumeta;https://twitter....
3      https://www.twitch.tv/1ief;https://twitter.com...
4      https://www.twitch.tv/1ndraking;https://twitte...
                             ...                        
271    https://www.twitch.tv/zombs;https://twitter.co...
272    https://www.twitch.tv/zondalol;None;http://fac...
273    https://www.twitch.tv/zoo9;https://twitter.com...
274    https://www.twitch.tv/zSxlem;https://twitter.c...
275                             None;None;None;None;None
Name: social_links, Length: 1276, dtype: object

**========= Player Details =========**

TO DO:

* FIX PLACEMENT COUNTS in getPlayerStats and getCoachStats
* FIX team_name EXCEPTION in getPlacement
* update getPlayerRecord to also create player activity dataframe

FILES:

* entity_details.csv
* player_activity.csv
* talent_activity.csv
* coach_activity.csv
* player transfers.csv
* event_details.csv

In [8]:
def calcYearsActive(active_string):
    active_list = active_string.split(';')
    years_active = int(active_list[-1].split('-')[0]) - int(active_list[0].split('-')[0])
    return years_active

In [32]:
def getPlacement(event_title,team_name,table):
    link = None
    if re.findall(fr'({event_title})',str(table)):
        I = re.search(fr'({event_title})',str(table)).start()
        base_link = 'https://liquipedia.net/apexlegends/'
        end_link = str(table)[I-(len(event_title)+10):I+(len(event_title)+10)].split('=')[1][1:].split('>')[0][:-1]
        link = base_link + end_link
    
    url = urllib.parse.urlsplit(link)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
   
    event_soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
    
    
    temp_table = event_soup.find_all("table",{"class":"table table-bordered prizepooltable collapsed"})
    
    for table in temp_table:
        temp_test = table.find_all("b")
        tier_list_index = [(m.start(0), m.end(0)) for m in re.finditer(r'(?:\d+th|\d+st|\d+rd|\d+nd)',str(temp_test))]
        tier_list = [int(re.sub(r'(th|st|rd|nd)', '', str(temp_test)[i_tuple[0]:i_tuple[-1]])) for i_tuple in tier_list_index]

        tier_max = max(tier_list)
        tier_list = list(range(1,tier_max+1))
        tier_list

        temp_df = pd.read_html(str(table), header=0)[0]
        temp_df['Team'].tolist()

        placement_list = {}
        for tier in tier_list:
            placement_list[temp_df['Team'].tolist()[tier-1]] = tier
        try:
            place = placement_list[team_name]
        except Exception as e:
            print('getPlacement',e)
            place = -1 

    return place

In [35]:
def getTalentStats(df):
    talent_total_events_covered = len(df)
    range_string = str(df['date'].iloc[-1]) + ';' + str(df['date'].iloc[0])
    return [talent_total_events_covered,range_string]

In [34]:
def getPlayerStats(df):
    total_winnings = df['winnings'].sum()
    df.loc[(df['placement'] == 'DQ'),'placement']=-1
    temp_df = df.loc[(df['placement'].astype(int) <= 3)]
    # FIX PLACEMENT COUNTS
    placement_counts = []
    temp_df.loc[:,['placement']] = temp_df['placement'].astype(int)
    placement_counts = temp_df['placement'].value_counts()
    try:
        placement_counts.sort_index(inplace=True)
    except Exception as e:
        print('getPlayerStats',e)
    range_string = str(df['date'].iloc[-1]) + ';' + str(df['date'].iloc[0])
    return [total_winnings,placement_counts,range_string]

In [33]:
def getCoachStats(df):
    total_winnings = df['winnings'].sum()
    df.loc[(df['placement'] == 'DQ'),'placement']=-1
    temp_df = df.loc[(df['placement'].astype(int) <= 3)]
    placement_counts = []
    placement_counts = temp_df['placement'].value_counts()
    try:
        placement_counts.sort_index(inplace=True)
        if len(placement_counts) > 3:
            combine_3 = placement_counts.iloc[2].astype(int) + placement_counts.iloc[-1].astype(int)
            placement_counts.iloc[2] = combine_3
            placement_counts.drop(placement_counts.iloc[-1], axis=0, inplace = True)
            placement_counts.sort_index(inplace=True)
    except Exception as e:
        print('getCoachStats',e)
    range_string = str(df['date'].iloc[-1]) + ';' + str(df['date'].iloc[0])
    return [total_winnings,placement_counts,range_string]

In [37]:
def getTalentActivity(link,talent_name):
    
    url = urllib.parse.urlsplit(link)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    
    try:
        talent_soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
        temp_table = talent_soup.find("table")
        temp_df = pd.read_html(str(temp_table))[0]
        
        #drop columns and clean up
        temp_df.drop(temp_df.columns[6:], axis = 1, inplace = True)
        temp_df.drop(columns=['Tournament'], inplace = True)
        temp_df.rename( columns={'Date':'date','Tier':'tier','Tournament.1':'event','Position':'position','Partner List':'partner_list'}, inplace=True )
        temp_df = temp_df[temp_df['tier'] != temp_df['position']]
        temp_df.reset_index(drop=True, inplace=True)
        
        # ISSUE SPOT
        reform_series = temp_df['partner_list'].str.split().str.join(';').astype(str)
        temp_df['partner_list'] = np.resize(reform_series,len(temp_df))
        
        temp_df.insert(0, 'talent_name', talent_name)
        temp_col = temp_df.pop('date')
        new_cols = list(temp_df.columns)
        new_cols.append('date')
        temp_df = temp_df.reindex(columns=new_cols)
        temp_df.loc[:,['date']] = temp_col
        temp_df.reset_index(drop=True, inplace=True)
        
        return temp_df
    except Exception as e:
        print('getTalentActivity',e)
        return pd.DataFrame()

In [38]:
def getPlayerActivity(link,player_name):
    
    url = urllib.parse.urlsplit(link)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    
    try:
        player_soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
        temp_table = player_soup.find("table")

        temp_td = temp_table.find_all("td",{"class":"results-team-icon"})
        temp_df = pd.read_html(str(temp_table), header=0)[0]
        temp_df = temp_df[temp_df['Tier'] != temp_df['Placement']]
        temp_df.rename( columns={'Date':'date','Placement':'placement','Tier':'tier','Tournament':'event','Team':'team','Prize':'winnings'}, inplace=True )
        temp_df.loc[:,['tier']] = temp_df['tier'].str.split('(').str[0].str.replace(' ','')
        temp_df.loc[:,['winnings']] = temp_df['winnings'].str.replace('$','',regex=True).str.replace(',','',regex=True).str.replace(' ','',regex=True).astype(float)
        
        
        temp_list = [str(item)[str(item).find('highlightingclass')+19:].split('>')[0][:-1] for item in temp_td] 
        temp_df.loc[:,['team']] = temp_list
        temp_df.insert(0, 'player_name', player_name)
        temp_df['placement'].replace(r"(th|st|rd|nd)", "", regex=True, inplace=True)

        temp_col = temp_df.pop('date')
        new_cols = list(temp_df.columns)
        new_cols.append('date')
        temp_df = temp_df.reindex(columns=new_cols)
        temp_df['date'] = temp_col
        temp_df.reset_index(drop=True, inplace=True)
        
        unknown_placements = temp_df['placement'].str.findall(r'\d+')
        temp_rows_index = list(unknown_placements.loc[unknown_placements.str.len() > 1].index)

        if temp_rows_index:
            temp_get_event_list = []
            for index in temp_rows_index:
                temp_get_event_list.append(temp_df['event'].iloc[index])

            temp_get_team_list = []
            for index in temp_rows_index:
                temp_get_team_list.append(temp_df['team'].iloc[index])

            placement_list = []
            for i in range(len(temp_get_event_list)):
                placement_list.append(getPlacement(temp_get_event_list[i],temp_get_team_list[i],temp_table))
            for i in range(len(placement_list)):
                temp_df.at[temp_rows_index[i],'placement'] = placement_list[i]
                
            
        return temp_df
    except Exception as e:
        print('getPlayerActivity',e)
        return pd.DataFrame()

In [39]:
def getCoachActivity(link,coach_name):
    
    url = urllib.parse.urlsplit(link)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    
    try:
        talent_soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
        temp_table = talent_soup.find("table")
        temp_td = temp_table.find_all("td",{"class":"results-team-icon"})
        temp_df = pd.read_html(str(temp_table), header=0)[0]
        
        #drop columns and clean up
        temp_df.rename( columns={'Date':'date','Placement':'placement','Tier':'tier','Tournament':'event','Team':'team','Prize':'winnings'}, inplace=True )
        temp_df = temp_df[temp_df['tier'] != temp_df['event']]
        temp_list = [str(item)[str(item).find('highlightingclass')+19:].split('>')[0][:-1] for item in temp_td] 
        temp_df.loc[:,['team']] = temp_list
        temp_df.loc[:,['tier']] = temp_df['tier'].str.split('(').str[0].str.replace(' ','')
        temp_df['placement'].replace(r"(th|st|rd|nd)", "", regex=True, inplace=True)
        temp_df.loc[:,['winnings']] = temp_df['winnings'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(float)
        temp_df.insert(0, 'coach_name', coach_name)
        
        temp_col = temp_df.pop('date')
        new_cols = list(temp_df.columns)
        new_cols.append('date')
        temp_df = temp_df.reindex(columns=new_cols)
        temp_df.loc[:,['date']] = temp_col
        temp_df.reset_index()
        
        
        
        unknown_placements = temp_df['placement'].str.findall(r'\d+')
        temp_rows_index = list(unknown_placements.loc[unknown_placements.str.len() > 1].index)

        if temp_rows_index:
            temp_get_event_list = []
            for index in temp_rows_index:
                temp_get_event_list.append(temp_df['event'].iloc[index])

            temp_get_team_list = []
            for index in temp_rows_index:
                temp_get_team_list.append(temp_df['team'].iloc[index])

            placement_list = []
            for i in range(len(temp_get_event_list)):
                placement_list.append(getPlacement(temp_get_event_list[i],temp_get_team_list[i],temp_table))
            for i in range(len(placement_list)):
                temp_df.at[temp_rows_index[i],'placement'] = placement_list[i]
        
        return temp_df
    except Exception as e:
        print('getCoachActivity',e)
        return pd.DataFrame()

In [40]:
def getEntityActivityAndStats(link_list,entity_name):
    player_link = link_list[0]
    talent_link = link_list[1]
    coach_link = link_list[2]
    
    entity_talent_stats_list = []
    entity_player_stats_list = []
    entity_coach_stats_list = []
    
    entity_talent_df = getTalentActivity(talent_link,entity_name)
    entity_player_df = getPlayerActivity(player_link,entity_name)
    entity_coach_df = getCoachActivity(coach_link,entity_name)
    
    if not entity_talent_df.empty: entity_talent_stats_list = getTalentStats(entity_talent_df)
    if not entity_player_df.empty: entity_player_stats_list = getPlayerStats(entity_player_df)
    if not entity_coach_df.empty: entity_coach_stats_list = getCoachStats(entity_coach_df)
        

    return entity_talent_df,entity_player_df,entity_coach_df,entity_talent_stats_list,entity_player_stats_list,entity_coach_stats_list

https://liquipedia.net/apexlegends/Android_Dream

In [42]:
def getEntityRecord(entity_name):
    entity_name = entity_name.replace(' ','_')
    base_entity_link = "https://liquipedia.net/apexlegends/"
    ind_entity_link = base_entity_link + entity_name
    
    player_link = f'https://liquipedia.net/apexlegends/{entity_name}/Results'
    talent_link = f'https://liquipedia.net/apexlegends/{entity_name}/Broadcasts'
    coach_link = f'https://liquipedia.net/apexlegends/{entity_name}/Coaching'
    link_list = [player_link,talent_link,coach_link]
    
    # USE URL FIX!!!
    url = urllib.parse.urlsplit(ind_entity_link)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    
    temp_soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
    temp_div = temp_soup.find("div",{"class":"fo-nttax-infobox wiki-bordercolor-light"})

    entity_info_box = temp_div.find_all("div", {"class":"infobox-cell-2"})

    # Collect birth date string and convert to pandas datetime object
    DOB = pd.NA
    if re.findall(r"(bday)",str(entity_info_box)):
        I = str(entity_info_box).find('bday')
        DOB = str(entity_info_box)[I:I+30].split('>')[1].split('<')[0]

    # Collect algs_points as an int
    algs_points_int = pd.NA
    if re.findall(r'(ALGS points)',str(entity_info_box)):
        I = str(entity_info_box).find('ALGS points')
        algs_points_int = int(str(entity_info_box)[I+62:].split('<')[0].replace(',',''))

    main_legends_string = pd.NA
    if re.findall(r'(Main Legends:)',str(entity_info_box)):
        legends_index = re.search(r'(Main Legends:)',str(entity_info_box)).start()
        legend_info = re.finditer(r'(title=)',str(entity_info_box)[legends_index:])
        all_legends_index = [legend.start() for legend in legend_info]
        all_legends_list = []
        for i in all_legends_index:
            all_legends_list.append(str(entity_info_box)[legends_index+i+7:].split('>')[0][:-1])
        main_legends_string = ';'.join(all_legends_list).lower()

    # Collect input as string
    input_string = pd.NA
    if re.findall(r'(Input:)',str(entity_info_box)):
        input_index = re.search(r'(Input:)',str(entity_info_box)).start()
        input_string = str(entity_info_box)[input_index+42:].split('<')[0].replace('\n','').replace(' ','').replace('&amp;',' & ').lower()
        
     # Collect alternate_ids as string with delimeter = ;
    alternate_ids_string = pd.NA
    if re.findall(r'(Alternate IDs:)',str(entity_info_box)):
        I = str(entity_info_box).find('Alternate IDs:')
        alternate_ids_string = str(entity_info_box)[I:].split('<')[2].split('>')[-1].replace(' ','').replace(',',';')

# ================================= NEED TO RETURN AND HANDLE DF's BEYOND SUMMARY STATS
    entity_talent_df,entity_player_df,entity_coach_df,entity_talent_stats_list,entity_player_stats_list,entity_coach_stats_list = getEntityActivityAndStats(link_list,entity_name)

    roles_list = []

    if (entity_player_stats_list) and (len(entity_player_stats_list) == 3):
        roles_list.append('player')
        player_total_winnings = entity_player_stats_list[0]
        player_placement_counts = entity_player_stats_list[1]
        player_range_string = entity_player_stats_list[2]
        player_years_active = calcYearsActive(player_range_string)
                
        try:
            player_placements_gold = player_placement_counts[1]
        except:
            player_placements_gold = 0
        try:
            player_placements_silver = player_placement_counts[2]
        except:
            player_placements_silver = 0
        try:
            player_placements_bronze = player_placement_counts[3]
        except:
            player_placements_bronze = 0
    else:
        player_total_winnings = pd.NA
        player_placements_gold = pd.NA
        player_placements_silver = pd.NA
        player_placements_bronze = pd.NA
        player_range_string = pd.NA
        player_years_active = pd.NA
        
    if (entity_talent_stats_list) and (len(entity_talent_stats_list) == 2):
        roles_list.append('talent')
        talent_events_covered = entity_talent_stats_list[0]
        talent_range_string = entity_talent_stats_list[1]
        talent_years_active = calcYearsActive(talent_range_string)
    else:
        talent_events_covered = pd.NA
        talent_range_string = pd.NA
        talent_years_active = pd.NA

    if (entity_coach_stats_list) and (len(entity_coach_stats_list) == 3):
        roles_list.append('coach') 
        coach_total_winnings = entity_coach_stats_list[0]
        coach_placement_counts = entity_coach_stats_list[1]
        coach_range_string = entity_coach_stats_list[2]    
        coach_years_active = calcYearsActive(coach_range_string)
        try:
            coach_placements_gold = coach_placement_counts[1]
        except:
            coach_placements_gold = 0
        try:
            coach_placements_silver = coach_placement_counts[2]
        except:
            coach_placements_silver = 0
        try:
            coach_placements_bronze = coach_placement_counts[3]
        except:
            coach_placements_bronze = 0
    else:
        coach_total_winnings = pd.NA
        coach_placements_gold = pd.NA
        coach_placements_silver = pd.NA
        coach_placements_bronze = pd.NA
        coach_range_string = pd.NA
        coach_years_active = pd.NA
        
    
    if roles_list:
        roles_string = ';'.join(roles_list)
    else:
        roles_string = pd.NA
    
    return entity_name,DOB,main_legends_string,input_string,roles_string,algs_points_int,alternate_ids_string,player_range_string,player_years_active,player_total_winnings,player_placements_gold,player_placements_silver,player_placements_bronze,talent_range_string,talent_years_active,talent_events_covered,coach_range_string,coach_years_active,coach_total_winnings,coach_placements_gold,coach_placements_silver,coach_placements_bronze

In [48]:
record = getEntityRecord('Gdolphn')

/home/skinofhotdog/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [49]:
record

('Gdolphn',
 '1995-04-11',
 'octane',
 'mouse & keyboard',
 'player;talent;coach',
 3600,
 'gdolphin',
 '2019-04-27;2022-03-15',
 3,
 35120.0,
 3,
 9,
 3,
 '2020-07-07;2022-01-10',
 2,
 21,
 '2020-10-02;2021-09-30',
 1,
 164843.0,
 8,
 7,
 0)

In [21]:
temp_name_list = ['ImperialHal','Hill','Aesir','Aeon','Dezignful','1000Eyes','Gemasoft','Daltoosh','Gdolphn']
# temp_name_list = ['ImperialHal','Daltoosh','Gdolphn']

In [50]:
R = []
I = 0

# for name in temp_name_list:
for name in all_entity_info['entity_name'].to_list():
    print(name)
    record = getEntityRecord(name)
    R.append(record)
    if (I % 50) == 0: os.system("windscribe connect")
    I += 1
    
os.system("windscribe disconnect")

0waRy
getTalentActivity HTTP Error 404: Not Found
getCoachActivity HTTP Error 404: Not Found


/home/skinofhotdog/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Connecting to US Central Dallas Ranch (UDP:443)
Firewall Enabled
Connected to US Central Dallas Ranch
Your IP changed from 155.94.217.4 to 107.150.30.86
1000Eyes
getPlayerActivity HTTP Error 404: Not Found
getCoachActivity HTTP Error 404: Not Found
1Fumeta
getTalentActivity HTTP Error 404: Not Found


/home/skinofhotdog/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


1ief
getTalentActivity HTTP Error 404: Not Found
getCoachActivity HTTP Error 404: Not Found
1ndraking


...

getTalentActivity HTTP Error 404: Not Found
getCoachActivity HTTP Error 404: Not Found
iShiny


/home/skinofhotdog/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


getTalentActivity HTTP Error 404: Not Found
getCoachActivity HTTP Error 404: Not Found
ISkyee


/home/skinofhotdog/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


ConnectionResetError: [Errno 104] Connection reset by peer

In [51]:
col_list = ['entity_id','dob','main_legends','input_type','roles',f'algs_points_{date.today().year-2}-{int(str(date.today().year-1)[-2:])}:','alternate_ids','player_years_active_range','player_years_active','player_total_winnings','player_placements_gold','player_placements_silver','player_placements_bronze','talent_years_active_range','talent_years_active','talent_events_covered','coach_years_active_range','coach_years_active','coach_total_winnings','coach_placements_gold','coach_placements_silver','coach_placements_bronze']

In [52]:
df = pd.DataFrame(R, columns=col_list)
# df[['entity_id','player_placements_gold','player_placements_silver','player_placements_bronze']]
df

,entity_id,dob,main_legends,input_type,roles,algs_points_2020-21:,alternate_ids,player_years_active_range,player_years_active,player_total_winnings,...,player_placements_bronze,talent_years_active_range,talent_years_active,talent_events_covered,coach_years_active_range,coach_years_active,coach_total_winnings,coach_placements_gold,coach_placements_silver,coach_placements_bronze
0,0waRy,08-01,<NA>,mouse & keyboard,player,1985,JINSEI0waRy,2020-04-18;2022-03-20,2,2697.0,...,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1000Eyes,<NA>,<NA>,mouse & keyboard,talent,<NA>,ThousandEyes,<NA>,<NA>,<NA>,...,<NA>,2021-05-09;2022-03-11,1,11,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,1Fumeta,<NA>,<NA>,mouse & keyboard,player;coach,<NA>,<NA>,2020-08-02;2021-05-16,1,579.0,...,1,<NA>,<NA>,<NA>,2020-12-04;2022-02-28,2,76883.0,5,3,0
3,1ief,1994-12-24,wraith;wattson;pathfinder,mouse & keyboard,player,6255,<NA>,2020-01-26;2021-12-05,1,2160.0,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1ndraking,1999-09-28,pathfinder;wraith,mouse & keyboard,player,4385,1NDR444,2019-11-23;2022-03-14,3,2920.0,...,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,Instyle,1994-07-03,wattson,mouse & keyboard,player,1460,<NA>,2019-10-31;2020-11-23,1,0.0,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
456,InvctuZ,<NA>,<NA>,controller,player,<NA>,<NA>,2021-04-01;2021-06-20,0,19922.0,...,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
457,IPirrus,2000-11-20,wraith,controller,player,1420,<NA>,2020-06-21;2022-02-15,2,8665.0,...,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
458,iPN,1995-07-26,bloodhound;revenant,mouse & keyboard,player,4965,iPN-k0u,2019-04-03;2022-01-14,3,66672.0,...,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [25]:
# df.insert(len(df.columns), 'social_links', all_entity_info['social_links'].to_list())

In [26]:
# General
print('entity_id:',record[0])
print('dob:',record[1])
print('main_legends:',record[2])
print('input_type:',record[3])
print('roles:',record[4])
print(f'algs_points_{date.today().year-2}-{int(str(date.today().year-1)[-2:])}:',record[5])
print('alternate_ids:',record[6])
# Player
print('player_years_active_range:',record[7])
print('player_years_active:',record[8])
print('player_total_winnings:',record[9])
print('player_placements_gold:',record[10])
print('player_placements_silver:',record[11])
print('player_placements_bronze:',record[12])
# Talent
print('talent_years_active_range:',record[13])
print('talent_years_active:',record[14])
print('talent_events_covered:',record[15])
# Coach
print('coach_years_active_range:',record[16])
print('coach_years_active:',record[17])
print('coach_total_winnings:',record[18])
print('coach_placements_gold:',record[19])
print('coach_placements_silver:',record[20])
print('coach_placements_bronze:',record[21])

entity_id: Gdolphn
dob: 1995-04-11
main_legends: octane
input_type: mouse & keyboard
roles: player;talent;coach
algs_points_2020-21: 3600
alternate_ids: gdolphin
player_years_active_range: 2019-04-27;2022-03-15
player_years_active: 3
player_total_winnings: 35120.0
player_placements_gold: 3
player_placements_silver: 9
player_placements_bronze: 3
talent_years_active_range: 2020-07-07;2022-01-10
talent_years_active: 2
talent_events_covered: 21
coach_years_active_range: 2020-10-02;2021-09-30
coach_years_active: 1
coach_total_winnings: 164843.0
coach_placements_gold: 8
coach_placements_silver: 7
coach_placements_bronze: 0


========= Tournaments =========